In [1]:
import pandas as pd

In [2]:
import numpy as np

In [2]:
import category_encoders as ce

In [3]:
from sklearn import preprocessing

In [6]:
df_events = pd.read_csv('data/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv', low_memory = False)

In [4]:
df_events.columns

Index(['timestamp', 'event', 'person', 'url', 'sku', 'model', 'condition',
       'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version', 'mes', 'dia', 'q1', 'q2',
       'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10', 'event_count',
       'tiempo_total', 'event_count_q1', 'event_count_q2', 'event_count_q3',
       'event_count_q4', 'event_count_q5', 'event_count_q6', 'event_count_q7',
       'event_count_q8', 'event_count_q9', 'event_count_q10',
       'tiempo_total_q1', 'tiempo_total_q2', 'tiempo_total_q3',
       'tiempo_total_q4', 'tiempo_total_q5', 'tiempo_total_q6',
       'tiempo_total_q7', 'tiempo_total_q8', 'tiempo_total_q9',
       'tiempo_total_q10'],
      dtype='object')

In [7]:
del(df_events)

In [8]:
encoder = ce.backward_difference.BackwardDifferenceEncoder(cols=['event'])

In [9]:
encoder.fit(data, verbose=1)

BackwardDifferenceEncoder(cols=['event'], drop_invariant=False,
             handle_unknown='impute', impute_missing=True,
             mapping=[{'col': 'event', 'mapping':        [D.1]     [D.2]     [D.3]     [D.4]     [D.5]     [D.6]     [D.7]  \
1  -0.909091 -0.818182 -0.727273 -0.636364 -0.545455 -0.454545 -0.363636
2   0.090909 -0.818182 -0.727273 -0.636364 -0.545455 -0.454545 -0.363636
3   0.090909  0.181818 -0.727273 -0...727273  0.818182 -0.090909
11  0.727273  0.818182  0.909091
0   0.000000  0.000000  0.000000  }],
             return_df=True, verbose=0)

In [10]:
event_transform = encoder.transform(data)

In [15]:
event_transform.shape

(2341681, 21)

In [16]:
event_transform.column

Index(['intercept', 'timestamp', 'event_0', 'event_1', 'event_2', 'event_3',
       'event_4', 'event_5', 'event_6', 'event_7', 'event_8', 'event_9',
       'person', 'sku', 'model', 'condition', 'storage', 'device_type',
       'screen_resolution', 'operating_system_version', 'browser_version'],
      dtype='object')

In [17]:
sum(pd.isnull(event_transform.storage))

1021151

In [18]:
sum(pd.isnull(event_transform.device_type))

2137612

In [19]:
sum(pd.isnull(event_transform.screen_resolution))

2137615

In [22]:
sum(pd.isnull(event_transform.operating_system_version))

2137612

In [21]:
sum(pd.isnull(event_transform.browser_version))

2137612

In [26]:
data = event_transform[['timestamp', 'event_0', 'event_1', 'event_2', 'event_3',
       'event_4', 'event_5', 'event_6', 'event_7', 'event_8', 'event_9',
       'person', 'sku', 'model', 'condition', 'storage']]

In [27]:
data.to_csv('dataEventBack.csv')

In [4]:
data = pd.read_csv('dataEventBack.csv')

/home/july/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# some date processing
def date_proc(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['year'] = df['timestamp'].dt.year
    df['month'] = df['timestamp'].dt.month
    df['day'] = df['timestamp'].dt.day
    df['weekday'] = df['timestamp'].dt.weekday
    df['hour'] = df['timestamp'].dt.hour
    df['year_month_day'] = df['timestamp'].map(lambda x: str(x.year)+"/"+str(x.month)+"/"+str(x.day))
    df['year_month_day'] = pd.to_datetime(df['year_month_day'])
    

In [6]:
date_proc(data)

In [7]:
from sklearn import preprocessing

In [8]:
weekday_le = preprocessing.LabelEncoder()
weekday_le.fit(data.weekday)

data.weekday = weekday_le.transform(data.weekday)

In [9]:
def month_to_cyclic(df):
    df['month_sin'] = df['month'].apply(lambda x: np.sin(2*np.pi*x/12))
    df['month_cos'] = df['month'].apply(lambda x: np.cos(2*np.pi*x/12))
    df.drop('month', axis=1, inplace=True)
    
def day_to_cyclic(df):
    df['day_sin'] = df['day'].apply(lambda x: np.sin(2*np.pi*x/31))
    df['day_cos'] = df['day'].apply(lambda x: np.cos(2*np.pi*x/31))
    df.drop('day', axis=1, inplace=True)

def weekday_to_cyclic(df):
    df['weekday_sin'] = df['weekday'].apply(lambda x: np.sin(2*np.pi*x/7))
    df['weekday_cos'] = df['weekday'].apply(lambda x: np.cos(2*np.pi*x/7))
    df.drop('weekday', axis=1, inplace=True)

def hour_to_cyclic(df):
    df['hour_sin'] = df['hour'].apply(lambda x: np.sin(2*np.pi*x/24))
    df['hour_cos'] = df['hour'].apply(lambda x: np.cos(2*np.pi*x/24))
    df.drop('hour', axis=1, inplace=True)

In [10]:
month_to_cyclic(data)
day_to_cyclic(data)
weekday_to_cyclic(data)


In [11]:
hour_to_cyclic(data)

In [12]:
data.columns

Index(['Unnamed: 0', 'timestamp', 'event_0', 'event_1', 'event_2', 'event_3',
       'event_4', 'event_5', 'event_6', 'event_7', 'event_8', 'event_9',
       'person', 'sku', 'model', 'condition', 'storage', 'year',
       'year_month_day', 'month_sin', 'month_cos', 'day_sin', 'day_cos',
       'weekday_sin', 'weekday_cos', 'hour_sin', 'hour_cos'],
      dtype='object')

In [17]:
del data['timestamp']
del data['year_month_day']
del data['person']


In [19]:
data.model = data.model.astype('category')

In [20]:
data.model.dtype

category

In [21]:
data.model = data.model.cat.codes

In [24]:
len(data.model.unique())

209

In [26]:
data.condition = data.condition.astype('category')

In [27]:
data.condition.unique()

[Excelente, Muito Bom, Bom, NaN, Bom - Sem Touch ID, Novo]
Categories (5, object): [Excelente, Muito Bom, Bom, Bom - Sem Touch ID, Novo]

In [28]:
conditions = {'condition': {'Excelente':4, 'Muito Bom': 3, 'Bom':2, 'NaN':0, 'Bom - Sem Touch ID':1, 'Novo':0}}

In [30]:
data.replace(conditions, inplace=True)

In [31]:
data.condition.unique()

array([ 4.,  3.,  2., nan,  1.,  0.])

In [36]:
data.fillna(value = {'condition' :0}, inplace=True)

,Unnamed: 0,event_0,event_1,event_2,event_3,event_4,event_5,event_6,event_7,event_8,...,storage,year,month_sin,month_cos,day_sin,day_cos,weekday_sin,weekday_cos,hour_sin,hour_cos
0,0,-0.909091,-0.818182,-0.727273,-0.636364,-0.545455,-0.454545,-0.363636,-0.272727,-0.181818,...,32GB,2018,0.5,-0.866025,-4.853020e-01,-0.874347,-0.433884,-0.900969,0.000000e+00,1.000000e+00
1,1,-0.909091,-0.818182,-0.727273,-0.636364,-0.545455,-0.454545,-0.363636,-0.272727,-0.181818,...,32GB,2018,0.5,-0.866025,-4.853020e-01,-0.874347,-0.433884,-0.900969,0.000000e+00,1.000000e+00
2,2,-0.909091,-0.818182,-0.727273,-0.636364,-0.545455,-0.454545,-0.363636,-0.272727,-0.181818,...,64GB,2018,0.5,-0.866025,-4.853020e-01,-0.874347,-0.433884,-0.900969,0.000000e+00,1.000000e+00
3,3,-0.909091,-0.818182,-0.727273,-0.636364,-0.545455,-0.454545,-0.363636,-0.272727,-0.181818,...,128GB,2018,0.5,-0.866025,-4.853020e-01,-0.874347,-0.433884,-0.900969,0.000000e+00,1.000000e+00
4,4,-0.909091,-0.818182,-0.727273,-0.636364,-0.545455,-0.454545,-0.363636,-0.272727,-0.181818,...,32GB,2018,0.5,-0.866025,-4.853020e-01,-0.874347,-0.433884,-0.900969,0.000000e+00,1.000000e+00
5,5,0.090909,-0.818182,-0.727273,-0.636364,-0.545455,-0.454545,-0.363636,-0.272727,-0.181818,...,NaN,2018,0.5,-0.866025,-4.853020e-01,-0.874347,-0.433884,-0.900969,0.000000e+00,1.000000e+00
6,6,-0.909091,-0.818182,-0.727273,-0.636364,-0.545455,-0.454545,-0.363636,-0.272727,-0.181818,...,16GB,2018,0.5,-0.866025,-4.853020e-01,-0.874347,-0.433884,-0.900969,0.000000e+00,1.000000e+00
7,7,-0.909091,-0.818182,-0.727273,-0.636364,-0.545455,-0.454545,-0.363636,-0.272727,-0.181818,...,128GB,2018,0.5,-0.866025,-4.853020e-01,-0.874347,-0.433884,-0.900969,0.000000e+00,1.000000e+00
8,8,-0.909091,-0.818182,-0.727273,-0.636364,-0.545455,-0.454545,-0.363636,-0.272727,-0.181818,...,32GB,2018,0.5,-0.866025,-4.853020e-01,-0.874347,-0.433884,-0.900969,0.000000e+00,1.000000e+00
9,9,0.090909,0.181818,-0.727273,-0.636364,-0.545455,-0.454545,-0.363636,-0.272727,-0.181818,...,NaN,2018,0.5,-0.866025,-4.853020e-01,-0.874347,-0.433884,-0.900969,0.000000e+00,1.000000e+00


In [33]:
data.storage.unique()

array(['32GB', '64GB', '128GB', nan, '16GB', '8GB', '256GB', '4GB',
       '512MB'], dtype=object)

In [38]:
storages = {'storage':{'32GB': 32, '64GB':64, '128GB':128, '16GB':16, '8GB':8, '256GB':256, '4GB':4,
       '512MB':512}}

In [39]:
data.replace(storages, inplace=True)

In [40]:
data.dtypes

Unnamed: 0       int64
event_0        float64
event_1        float64
event_2        float64
event_3        float64
event_4        float64
event_5        float64
event_6        float64
event_7        float64
event_8        float64
event_9        float64
sku            float64
model            int16
condition      float64
storage        float64
year             int64
month_sin      float64
month_cos      float64
day_sin        float64
day_cos        float64
weekday_sin    float64
weekday_cos    float64
hour_sin       float64
hour_cos       float64
dtype: object

In [1]:
import pickle

In [47]:
filename = 'data.pkl'
with open(filename,'wb') as file:
    pickle.dump(data, file)

In [4]:
filename = 'data.pkl'
with open(filename, 'rb') as file:
    data = pickle.load(file)

In [5]:
data.columns

Index(['Unnamed: 0', 'event_0', 'event_1', 'event_2', 'event_3', 'event_4',
       'event_5', 'event_6', 'event_7', 'event_8', 'event_9', 'sku', 'model',
       'condition', 'storage', 'year', 'month_sin', 'month_cos', 'day_sin',
       'day_cos', 'weekday_sin', 'weekday_cos', 'hour_sin', 'hour_cos'],
      dtype='object')

In [48]:
data.to_csv('data/dataJuli.csv')

In [7]:
data['person'] = df_events['person']

NameError: name 'data' is not defined

In [7]:
del df_events

In [8]:
data.columns

Index(['Unnamed: 0', 'event_0', 'event_1', 'event_2', 'event_3', 'event_4',
       'event_5', 'event_6', 'event_7', 'event_8', 'event_9', 'sku', 'model',
       'condition', 'storage', 'year', 'month_sin', 'month_cos', 'day_sin',
       'day_cos', 'weekday_sin', 'weekday_cos', 'hour_sin', 'hour_cos',
       'person'],
      dtype='object')

In [9]:
label = pd.read_csv('data/fiuba-trocafone-tp2-final-set/labels_training_set.csv')

In [10]:
label.columns

Index(['person', 'label'], dtype='object')

In [11]:
train = pd.merge(data, label, on = 'person', how = 'right')

In [12]:
train.columns

Index(['Unnamed: 0', 'event_0', 'event_1', 'event_2', 'event_3', 'event_4',
       'event_5', 'event_6', 'event_7', 'event_8', 'event_9', 'sku', 'model',
       'condition', 'storage', 'year', 'month_sin', 'month_cos', 'day_sin',
       'day_cos', 'weekday_sin', 'weekday_cos', 'hour_sin', 'hour_cos',
       'person', 'label'],
      dtype='object')

In [59]:
train.shape

(1171886, 26)

In [13]:
del train['Unnamed: 0']
del train['person']

In [9]:
filename = 'train_back.pkl'
with open(filename,'wb') as file:
    pickle.dump(train, file)

In [2]:
import pickle

In [2]:
filename = 'train_back.pkl'
with open(filename,'rb') as file:
    train = pickle.load(file)

In [11]:
del df_events

In [10]:
train.columns

Index(['event_0', 'event_1', 'event_2', 'event_3', 'event_4', 'event_5',
       'event_6', 'event_7', 'event_8', 'event_9', 'sku', 'model', 'condition',
       'storage', 'year', 'month_sin', 'month_cos', 'day_sin', 'day_cos',
       'weekday_sin', 'weekday_cos', 'hour_sin', 'hour_cos', 'label',
       'person'],
      dtype='object')

In [8]:
train['person'] = df_events.person

In [10]:
label_test = pd.read_csv('data/trocafone_kaggle_test.csv')

In [15]:
test = pd.merge(data, label_test, on='person', how='right')

In [16]:
test.shape

(1169795, 25)

In [17]:
del data

In [18]:
filename = 'test_back.pkl'
with open(filename,'wb') as file:
    pickle.dump(test, file)

In [7]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold


In [ ]:
model = XGBRegressor(learning_rate=0.01, n_estimators=1000)

model.fit(X_train, y_train)

In [19]:
from xgboost import XGBRegressor


In [22]:
train.columns

Index(['event_0', 'event_1', 'event_2', 'event_3', 'event_4', 'event_5',
       'event_6', 'event_7', 'event_8', 'event_9', 'sku', 'model', 'condition',
       'storage', 'year', 'month_sin', 'month_cos', 'day_sin', 'day_cos',
       'weekday_sin', 'weekday_cos', 'hour_sin', 'hour_cos', 'label'],
      dtype='object')

In [15]:
X_train = train[['event_0', 'event_1', 'event_2', 'event_3', 'event_4', 'event_5',
       'event_6', 'event_7', 'event_8', 'event_9', 'sku', 'model', 'condition',
       'storage', 'year', 'month_sin', 'month_cos', 'day_sin', 'day_cos',
       'weekday_sin', 'weekday_cos', 'hour_sin', 'hour_cos']]

In [16]:
y_train = train.label

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =  train_test_split(X_train, y_train, test_size=0.3, random_state=100)

In [20]:
xgb = XGBRegressor(n_estimators=100, max_depth=3)

In [21]:
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [23]:
preds = xgb.predict(test)

In [24]:
preds

array([0.05567372, 0.05567372, 0.04489422, ..., 0.15574288, 0.04820439,
       0.08235687], dtype=float32)

In [ ]:
preds.to_csv('predXg')

In [26]:
filename = 'test_back.pkl'
with open(filename,'rb') as file:
    test = pickle.load(file)

In [13]:
test.columns

Index(['Unnamed: 0', 'event_0', 'event_1', 'event_2', 'event_3', 'event_4',
       'event_5', 'event_6', 'event_7', 'event_8', 'event_9', 'sku', 'model',
       'condition', 'storage', 'year', 'month_sin', 'month_cos', 'day_sin',
       'day_cos', 'weekday_sin', 'weekday_cos', 'hour_sin', 'hour_cos',
       'person'],
      dtype='object')

In [32]:
pred = xgb.predict_proba(X_test)

In [31]:
from sklearn.metrics import roc_auc_score

In [37]:
pred[:,1]

array([0.10963181, 0.12556489, 0.1091906 , ..., 0.10963181, 0.13910997,
       0.12556489], dtype=float32)

In [38]:
roc_auc_score(pred[:,1], y_test)

ValueError: continuous format is not supported

In [14]:
del test['person']
del test['Unnamed: 0']

In [43]:
filename = 'xg_back.pkl'
with open(filename,'wb') as file:
    pickle.dump(xgb, file)

In [3]:
filename = 'xg_back.pkl'
with open(filename,'rb') as file:
    xgb = pickle.load(file)

In [22]:
test.columns

Index(['event_0', 'event_1', 'event_2', 'event_3', 'event_4', 'event_5',
       'event_6', 'event_7', 'event_8', 'event_9', 'sku', 'model', 'condition',
       'storage', 'year', 'month_sin', 'month_cos', 'day_sin', 'day_cos',
       'weekday_sin', 'weekday_cos', 'hour_sin', 'hour_cos'],
      dtype='object')

In [22]:
del test['Unnamed: 0']

In [23]:
pred_posta = xgb.predict_proba(test)

In [27]:
len(pd.unique(pred_posta[:,1]))

2073

In [28]:
len(pred_posta)

1169795

In [29]:
pred_posta

array([[0.89036816, 0.10963181],
       [0.89036816, 0.10963181],
       [0.89036816, 0.10963181],
       ...,
       [0.81209254, 0.18790746],
       [0.854892  , 0.14510798],
       [0.854892  , 0.14510798]], dtype=float32)

In [28]:
to_publish = pd.DataFrame()

In [30]:
filename = 'test_back.pkl'
with open(filename,'rb') as file:
    test = pickle.load(file)

In [29]:
to_publish['person'] = test.person

In [30]:
to_publish['label'] = preds

In [31]:
to_publish.shape

(1169795, 2)

In [32]:
a = to_publish.groupby('person', as_index=False).mean()

In [ ]:
a

In [36]:
a.to_csv('predictions/xgbJuli.csv',index=False)

In [42]:
to_publish.to_csv('mala_prediccion.csv')